# Data preparation

In [ ]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the round
my_round = "fifth_round"

# read the tables
rater_one = pd.read_excel('/content/drive/MyDrive/CHR2024/Annotations/'+my_round+'_ann1.xlsx', index_col=0)
rater_two = pd.read_excel('/content/drive/MyDrive/CHR2024/Annotations/'+my_round+'_ann2.xlsx', index_col=0)

# finding the portion of the dataset tagged entirely by both
tagged_untill = []
for df in [rater_one, rater_two]:
  for i in range(df.shape[0]):
    if all(pd.isna(x) for x in df['label'].tolist()[i:]):
      tagged_untill.append(i)
      break

if len(tagged_untill) > 0:
  cut = min(tagged_untill)
else:
  cut = rater_one.shape[0]

print("Annotated sentences:")
print(cut)

# convert to cat.codes (to find NAs)
rater_one_annotations = rater_one[:cut]['label'].astype('category').cat.codes
rater_two_annotations = rater_two[:cut]['label'].astype('category').cat.codes

# find NAs (sentences/reviews not annotated: this needs to be avoided!)
nan_indexes_one = [index for index, value in enumerate(rater_one_annotations) if value == -1]
nan_indexes_two = [index for index, value in enumerate(rater_two_annotations) if value == -1]
nan_indexes = nan_indexes_one + nan_indexes_two
nan_indexes = list(set(nan_indexes))

print("Missed sentences:")
print(len(nan_indexes))
if len(nan_indexes) > 0:
  print("rater_one:")
  print(nan_indexes_one)
  print("rater_two:")
  print(nan_indexes_two)

Annotated sentences:
2056
Missed sentences:
0


# Stats

Print stats of label usage

In [ ]:
rater_one_stats = rater_one[:cut].groupby('label').size().reset_index(name='Count')
rater_two_stats = rater_two[:cut].groupby('label').size().reset_index(name='Count')
stats_df = pd.merge(rater_one_stats, rater_two_stats, on='label', suffixes=('_rater_one', '_rater_two'))
stats_df

,label,Count_rater_one,Count_rater_two
0,aesthetic,278,245
1,generic_val,127,96
2,ind_cognitive,7,4
3,ind_emotional,66,84
4,no_val,1573,1608
5,social,5,1


Kappa metrics

In [ ]:
all_kappas = []

# read annotations
rater_one_annotations = rater_one[:cut]['label']
rater_two_annotations = rater_two[:cut]['label']

# remove Nas (if any)
for index in sorted(nan_indexes, reverse=True):
   del rater_one_annotations[index]
   del rater_two_annotations[index]

# Calculate Cohen's Kappa
kappa = cohen_kappa_score(rater_one_annotations, rater_two_annotations)
all_kappas.extend([kappa])

all_labels = ['no_val', 'aesthetic', 'ind_cognitive', 'ind_emotional', 'ind_pragmatic', 'social', 'generic_val']

for my_cat in all_labels:

  # normalize (to val vs. no_val)
  rater_one_annotations_my_cat = ["other" if value != my_cat else value for value in rater_one_annotations]
  rater_two_annotations_my_cat = ["other" if value != my_cat else value for value in rater_two_annotations]

  # Calculate Cohen's Kappa
  kappa = cohen_kappa_score(rater_one_annotations_my_cat, rater_two_annotations_my_cat)

  all_kappas.extend([kappa])

kappas_df = pd.DataFrame({"label":["all_labels"]+all_labels, "kappa_"+my_round:all_kappas})

# calulate kappas from previous rounds for comparison
previous_rounds = ["fourth_round", "third_round", "second_round"]

for my_previous_round in previous_rounds:

  all_kappas = []

  # read the tables
  rater_one_previous = pd.read_excel('/content/drive/MyDrive/CHR2024/Annotations/'+my_previous_round+'_ann1.xlsx', index_col=0)
  rater_two_previous = pd.read_excel('/content/drive/MyDrive/CHR2024/Annotations/'+my_previous_round+'_ann2.xlsx', index_col=0)

  # read annotations
  rater_one_annotations = rater_one_previous['label']
  rater_two_annotations = rater_two_previous['label']

  # Calculate Cohen's Kappa
  kappa = cohen_kappa_score(rater_one_annotations, rater_two_annotations)
  all_kappas.extend([kappa])

  for my_cat in all_labels:

    # normalize (to val vs. no_val)
    rater_one_annotations_my_cat = ["other" if value != my_cat else value for value in rater_one_annotations]
    rater_two_annotations_my_cat = ["other" if value != my_cat else value for value in rater_two_annotations]

    # Calculate Cohen's Kappa
    kappa = cohen_kappa_score(rater_one_annotations_my_cat, rater_two_annotations_my_cat)

    all_kappas.extend([kappa])

  kappas_df["kappa_"+my_previous_round] = all_kappas

kappas_df

,label,kappa_fifth_round,kappa_fourth_round,kappa_third_round,kappa_second_round
0,all_labels,0.570145,0.519331,0.460970,0.499723
1,no_val,0.648860,0.648603,0.576418,0.663655
2,aesthetic,0.529278,0.497913,0.521532,0.437949
3,ind_cognitive,-0.002482,0.169788,0.187178,0.255142
4,ind_emotional,0.571252,0.275554,0.059792,0.397541
5,ind_pragmatic,0.000000,0.050479,0.178430,0.254805
6,social,0.332792,-0.001129,-0.005158,-0.002956
7,generic_val,0.474283,0.449889,0.329573,0.352385
